# Building the design matrix for TBSS

Modeling the interaction between ApoE and centiloids on DWI

In [82]:
%run /home/grg/git/alfa/nilearn-helper.py
%run /home/grg/git/alfa/build_matrix.py
%matplotlib inline

In [5]:
import json, string
import os.path as op
from glob import glob
dd = '/home/grg/SPM/2018'
data_dir = '/home/grg/TBSS/AMYLOID/amylstaging/origdata'
subjects = sorted([string.atoi(op.basename(e).split('_')[0]) for e in glob(op.join(data_dir, '*.nii.gz'))]) #.difference(set(failed)))


In [85]:
variables = ['age', 'gender', 'centiloids', 'apoe']
cov = pd.DataFrame(pd.read_excel(osp.join(dd, 'covariates.xlsx'), converters={'subject':int, 'apoe':int}), columns=variables)
cov = build_interaction(cov, 'centiloids', 'apoe')
del cov['centiloids']
cov.head()
#del data['stage']
contrasts = tbss_covariates_simple_contrasts(cov.columns)
contrasts_2vs2 = tbss_2vs2_contrasts(['centiloids0', 'centiloids1', 'centiloids2'], list(cov.columns))
contrasts.extend(contrasts_2vs2)
mat = build_tbss_matrix(cov)
con = build_tbss_contrasts(contrasts)
dump(contrasts, ['/tmp/con%s.con'%i for i in xrange(len(contrasts)/2)])

# writing files
w = open('/tmp/amylstaging_interaction_apoe_centiloids.mat','w')
w.write(mat)
w.close()
         

In [95]:
commands = []
def dump_commands(commands, fp):
    def chunks(l, n): 
        for i in range(0, len(l), n): 
            yield l[i:i + n]
    for each, f in zip(chunks(commands, len(commands)/len(fp)), fp):
        print each, f
        w = open(f, 'w')
        for e in each:
            w.write('%s\n'%e)
        w.close()

for t in ['MD', 'FA']:
    for i in xrange(len(contrasts)/2):        
        cmd = 'mailx -s "%s - %s - con%s - tbss_%s_ApoeCentilInteraction_con%s" < /dev/null "goperto@barcelonabeta.org"'%(t, contrasts[i][0], i, t, i)
        #commands.append(cmd)
        print cmd
        #cmd = 'randomise -i all_%s_skeletonised -o tbss_%s_ApoeCentilInteraction_con%s -m mean_FA_skeleton_mask -d interaction/amylstaging_interaction_apoe_centiloids.mat -t interaction/%s -n 1000 --T2 -V'\
        #    %(t, t, i, 'con%s.con'%i)
        #commands.append(cmd)

#dump_commands(commands, ['/tmp/script%s.sh'%i for i in xrange(7)])

mailx -s "MD - age(+) - con0 - tbss_MD_ApoeCentilInteraction_con0" < /dev/null "goperto@barcelonabeta.org"
mailx -s "MD - age(-) - con1 - tbss_MD_ApoeCentilInteraction_con1" < /dev/null "goperto@barcelonabeta.org"
mailx -s "MD - gender(+) - con2 - tbss_MD_ApoeCentilInteraction_con2" < /dev/null "goperto@barcelonabeta.org"
mailx -s "MD - gender(-) - con3 - tbss_MD_ApoeCentilInteraction_con3" < /dev/null "goperto@barcelonabeta.org"
mailx -s "MD - apoe(+) - con4 - tbss_MD_ApoeCentilInteraction_con4" < /dev/null "goperto@barcelonabeta.org"
mailx -s "MD - apoe(-) - con5 - tbss_MD_ApoeCentilInteraction_con5" < /dev/null "goperto@barcelonabeta.org"
mailx -s "MD - centiloids0(+) - con6 - tbss_MD_ApoeCentilInteraction_con6" < /dev/null "goperto@barcelonabeta.org"
mailx -s "MD - centiloids0(-) - con7 - tbss_MD_ApoeCentilInteraction_con7" < /dev/null "goperto@barcelonabeta.org"
mailx -s "MD - centiloids1(+) - con8 - tbss_MD_ApoeCentilInteraction_con8" < /dev/null "goperto@barcelonabeta.org"
mailx

In [89]:
len(commands)

48